# Streamlit App

## Importación de librerías

In [1]:
# Librerías básicas a descargar
import os
import pandas as pd
import numpy as np

# Librerías relacionadas con modelos
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Librerías relacionadas con aplicación
import streamlit as st
import folium # biblioteca para mapas interactivos
from streamlit_folium import st_folium

## Descargamos los datos

In [6]:
# Rutas (cambiarla por usuario)
pth2 = "../Gorrapiedra-proyecto/data/formated data/"
info_station2 = "../Gorrapiedra-proyecto/data/info bicing/Informacio_Estacions_Bicing.csv"

# Cargar los datos
df_20 = pd.read_csv(os.path.join(pth2, "data_2020.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_21 = pd.read_csv(os.path.join(pth2, "data_2021.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_22 = pd.read_csv(os.path.join(pth2, "data_2022.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_23 = pd.read_csv(os.path.join(pth2, "data_2023.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_info = pd.read_csv(os.path.join(info_station2), index_col=False, skipinitialspace=True, skip_blank_lines=True)

# Estaciones comunes (Filtrado)
dfs_list = [df_20, df_21, df_22, df_23]
for proto_df in dfs_list:
    for otro_df in dfs_list:
        if proto_df is not otro_df:
            proto_df = proto_df[proto_df['station_id'].isin(otro_df['station_id'])]

# Columnas de interés
df_info_stations = df_info[['station_id', 'lat', 'lon', 'post_code', 'capacity']]

# Concatenar en un mismo dataframe
df = pd.concat([df_20, df_21, df_22, df_23], ignore_index=True)
df = pd.merge(df, df_info_stations, on='station_id')

# Eliminar columnas innecesarias
if 'Unnamed: 0' in df.columns:
    df = df.drop(['Unnamed: 0'], axis=1)

print("min and max value of station id: ", df['station_id'].min(), df['station_id'].max())

# Visualizar el df
print("Shape of the df: ", df.shape)
df.head()

min and max value of station id:  1.0 519.0
Shape of the df:  (3275160, 14)


,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,percentage_docks_available,lat,lon,post_code,capacity
0,1.0,2020.0,1.0,1.0,2.0,0.522727,0.503788,0.469697,0.403409,0.354167,41.397978,2.180107,8013,45
1,1.0,2020.0,1.0,1.0,7.0,0.289773,0.373106,0.304924,0.238636,0.240530,41.397978,2.180107,8013,45
2,1.0,2020.0,1.0,1.0,12.0,0.253788,0.268939,0.350379,0.344697,0.393939,41.397978,2.180107,8013,45
3,1.0,2020.0,1.0,1.0,17.0,0.390152,0.346591,0.255814,0.220930,0.186047,41.397978,2.180107,8013,45
4,1.0,2020.0,1.0,1.0,22.0,0.083333,0.218992,0.437984,0.515504,0.470930,41.397978,2.180107,8013,45


## Modelo de regresión linear de prueba

In [3]:
# Selección de features y target
X = df[['year', 'month', 'day', 'hour', 'lat', 'lon', 'capacity']]
y = df['percentage_docks_available']

# Dividir los datos en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluación del modelo
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.07071574027882695


## Creamos Aplicación de Streamlit

In [7]:
# Título 
st.title("Bicing App Postgrau")

# Filtrar datos 
df_sample = df.sample(500)

# Creación de mapa de Barcelona
barcelona_map = [41.3851, 2.1734]
map = folium.Map(location=barcelona_map, zoom_start=12)

# Función para predicción de bicicletas disponibles
def predict_availability(row):
    features = np.array([[row['year'], row['month'], row['day'], row['hour'], row['lat'], row['lon'], row['capacity']]])
    return model.predict(features)[0]

# Marcadores al mapa (mostrar predicciones en las estaciones a partir de un marcador)
for idx, row in df_sample.iterrows():
    pred = predict_availability(row)
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Station ID: {row['station_id']}<br>Prediction: {pred:.2f}% of available docks",
        icon=folium.Icon(color="blue" if pred > 50 else "green")
    ).add_to(map)

# Visualización del mapa en la aplicación de Streamlit
st_folium(map, width=725, height=500)

# Texto decorativo sobre la aplicación
st.write("""Aquest mapa mostra la predicció dels espais buits de bicicletes a cadascuna de les estacions""")

c:\Users\34627\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\34627\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\34627\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\34627\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\34627\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\34627\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not ha

In [8]:
map

# EJEMPLO RANDOM DE CÓMO FUNCIONA LA LIBRERÍA FOLIUM (LO HE HECHO PARA ENTENDER MEJOR QUÉ ES)

In [9]:
import folium

# Crear un mapa
mapa = folium.Map(location=[41.38, 2.17], zoom_start=12)

# Marcador random
folium.Marker([41.38, 2.17], popup='Barcelona').add_to(mapa)

# Visualización
mapa

In [ ]:
# falta poner en terminal el siguiente código para poder ejecutar Streamlit app - sino probar con librería de visualización Voila
streamlit run view.py # para poder ejecutarlo debemos usar un archivo .py NO de Jupiter